In [ ]:
from ultralytics import YOLO
import sys
print("Python path:", sys.executable)
print("Python version:", sys.version)

#Carica il modello preaddestrato più piccolo
model = YOLO("yolo11l.pt")

# Esporta in formato TFLite quantizzato 
model.export(format='tflite', int8=True)

In [1]:
# --- Funzione di preprocess immagine MODIFICATA per salvare l'immagine ---
def preprocess_image(img_path, save_preprocessed=True):
    # Carica immagine con PIL (RGB)
    img = Image.open(img_path).convert('RGB')

    # Immagine originale dimensioni 640x480
    original_width, original_height = img.size
    print(f"Original image size: {original_width}x{original_height}")

    # Target size modello: 640x640
    target_size = 640

    # Crea canvas nero quadrato 640x640
    new_img = Image.new('RGB', (target_size, target_size), (0, 0, 0))

    # Calcola offset per centrare l'immagine originale sul canvas
    offset_x = 0
    offset_y = (target_size - original_height) // 2  # vertical padding

    # Incolla immagine originale al centro (horizontal no padding)
    new_img.paste(img, (offset_x, offset_y))

    # Salva l'immagine preprocessata se richiesto
    if save_preprocessed:
        preprocessed_path = img_path.replace('.jpg', '_preprocessed.jpg').replace('.png', '_preprocessed.png')
        new_img.save(preprocessed_path)
        print(f"Immagine preprocessata salvata come: {preprocessed_path}")

    # Converti in numpy array float32 e normalizza (se modello vuole 0-1)
    input_array = np.array(new_img).astype(np.float32) / 255.0

    # Aggiungi dimensione batch (1, 640, 640, 3)
    input_array = np.expand_dims(input_array, axis=0)

    return input_array


In [2]:
def preprocess_image_320(img_path, save_preprocessed=True):
    # Carica immagine con PIL (RGB) - già 320x320
    img = Image.open(img_path).convert('RGB')
    
    print(f"Immagine originale: {img.size}")
    
    # Salva se richiesto (opzionale, per debug)
    if save_preprocessed:
        preprocessed_path = img_path.replace('.jpg', '_preprocessed_320.jpg')
        img.save(preprocessed_path)
        print(f"Immagine preprocessata salvata come: {preprocessed_path}")
    
    # Solo normalizzazione e formato - NESSUN RESIZE
    input_array = np.array(img).astype(np.float32) / 255.0
    input_array = np.expand_dims(input_array, axis=0)  # (1, 320, 320, 3)
    
    return input_array

In [61]:
import numpy as np

#Funzione di post-processing
def detect_person(output_data, confidence_threshold=0.5, iou_threshold=0.5):
    """
    Post-processing per rilevare persone nell'immagine
    """
    # COCO class ID per 'person' è 0
    PERSON_CLASS_ID = 0 #dovrebbe essere 0 di default
    
    # Estrai le predizioni (rimuovi dimensione batch)
    predictions = output_data[0]  # Shape: (84, 8400)
    
    # Separa coordinate, confidence e classi
    # Ogni colonna: [x, y, w, h, confidence, class1, class2, ..., class80]
    boxes = predictions[:4, :].T  # (8400, 4) - coordinate relative
    confidences = predictions[29, :]  # (8400,) - confidence scores
    class_scores = predictions[5:, :].T  # (8400, 79) - class scores
    
    # Trova le detections per la classe 'person'
    person_scores = class_scores[:, PERSON_CLASS_ID]  # (8400,)
    
    # Combina confidence generale con confidence specifica della classe
    final_scores = confidences * person_scores
    
    # Filtra detections con score alto
    high_score_indices = np.where(final_scores > confidence_threshold)[0]
    
    if len(high_score_indices) == 0:
        print("❌ Nessuna persona rilevata nell'immagine")
        return []
    
    # Estrai bounding boxes e scores
    detected_boxes = []
    detected_scores = []
    
    for idx in high_score_indices:
        # Converti coordinate relative in pixel
        x_center, y_center, width, height = boxes[idx]
        
        # Converti da formato YOLO (center, width, height) a formato pixel (x1, y1, x2, y2)
        x1 = int((x_center - width/2) * 640)
        y1 = int((y_center - height/2) * 640)
        x2 = int((x_center + width/2) * 640)
        y2 = int((y_center + height/2) * 640)
        
        # Clamp ai bordi dell'immagine
        x1 = max(0, min(x1, 640))
        y1 = max(0, min(y1, 640))
        x2 = max(0, min(x2, 640))
        y2 = max(0, min(y2, 640))
        
        detected_boxes.append([x1, y1, x2, y2])
        detected_scores.append(final_scores[idx])
    
    # Non-maximum suppression per rimuovere duplicati
    if len(detected_boxes) > 1:
        # Implementazione semplificata di NMS
        keep_indices = []
        for i in range(len(detected_boxes)):
            keep = True
            for j in range(len(detected_boxes)):
                if i != j and detected_scores[j] > detected_scores[i]:
                    # Calcola IoU
                    box1 = detected_boxes[i]
                    box2 = detected_boxes[j]
                    
                    # Calcola area di intersezione
                    x1 = max(box1[0], box2[0])
                    y1 = max(box1[1], box2[1])
                    x2 = min(box1[2], box2[2])
                    y2 = min(box1[3], box2[3])
                    
                    if x2 > x1 and y2 > y1:
                        intersection = (x2 - x1) * (y2 - y1)
                        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
                        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
                        union = area1 + area2 - intersection
                        iou = intersection / union
                        
                        if iou > iou_threshold:
                            keep = False
                            break
            
            if keep:
                keep_indices.append(i)
        
        detected_boxes = [detected_boxes[i] for i in keep_indices]
        detected_scores = [detected_scores[i] for i in keep_indices]
    
    # Stampa risultati
    print(f"✅ Trovate {len(detected_boxes)} persona/e nell'immagine:")
    for i, (box, score) in enumerate(zip(detected_boxes, detected_scores)):
        x1, y1, x2, y2 = box
        print(f"   Persona {i+1}: Bounding box ({x1}, {y1}, {x2}, {y2}) - Confidence: {score:.3f}")
    
    return detected_boxes, detected_scores

In [153]:
from PIL import Image, ImageDraw, ImageFont

def draw_bounding_boxes(image_path, detected_boxes, detected_scores, output_path=None):
    """
    Disegna le bounding box sull'immagine e salvala
    """
    # Carica l'immagine preprocessata
    img = Image.open(image_path)
    
    # Crea un oggetto per disegnare
    draw = ImageDraw.Draw(img)
    
    # Colori per le bounding box (rosso per le persone)
    box_color = (255, 0, 0)  # Rosso
    text_color = (255, 255, 255)  # Bianco
    
    # Disegna ogni bounding box
    for i, (box, score) in enumerate(zip(detected_boxes, detected_scores)):
        x1, y1, x2, y2 = box
        
        # Disegna il rettangolo
        draw.rectangle([x1, y1, x2, y2], outline=box_color, width=3)
        
        # Aggiungi testo con confidence score
        text = f"Person {i+1}: {score:.3f}"
        
        # Posizione del testo (sopra la bounding box)
        text_x = x1
        text_y = max(0, y1 - 20)  # 20 pixel sopra la box
        
        # Disegna sfondo per il testo
        text_bbox = draw.textbbox((text_x, text_y), text)
        draw.rectangle([text_bbox[0]-2, text_bbox[1]-2, text_bbox[2]+2, text_bbox[3]+2], 
                      fill=box_color)
        
        # Disegna il testo
        draw.text((text_x, text_y), text, fill=text_color)
    
    # Salva l'immagine
    if output_path is None:
        output_path = image_path.replace('.jpg', '_with_boxes.jpg').replace('.png', '_with_boxes.png')
    
    img.save(output_path)
    print(f"Immagine con bounding box salvata come: {output_path}")
    
    return output_path

In [ ]:
#Riscriviamo a mano la cella di inferenza

In [170]:
#Proviamo a fare inferenza col modello quantizzato usando TensorFlow Lite
import tensorflow as tf
import numpy as np
from PIL import Image

#parametri modificabili per esperimenti


#nomeModello = "Modelli/Utili/yolo11n_int8.tflite"
#nomeModello = "Modelli/Utili/yolo11n_float32.tflite"
nomeModello = "Modelli/Utili/yolo11l_float32.tflite"
#nomeModello = "Modelli/Utili/yolo11n_float16.tflite"
#nomeModello = "yolo11n_int8personal.tflite"
nomeImmagine = "Foto/example"
confidence_threshold = 0.000000001 #confidence in base a cui detecti o meno qualcosa
iou_confidence = 0.1 #Intersection over Union, più è basso e più le bounding box contenute da altre, non sono considerate

# Carica il modello
interpreter = tf.lite.Interpreter(model_path=nomeModello)
interpreter.allocate_tensors()

# Ottieni info su input/output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Dettagli input modello:", input_details)
print("Dettagli output modello:",output_details)

#stampa formati input/output
print("formato dell'input:", input_details[0]['dtype'])
print("formato output:", output_details[0]['dtype'])

# --- Carica e preprocessa immagine ---
nomeImmagineJpg= nomeImmagine + ".jpg"
input_data = preprocess_image(nomeImmagineJpg)

# --- QUANTIZZAZIONE INPUT ---
if input_details[0]['dtype'] == np.int8:
    # Ottieni i parametri di quantizzazione dell'input
    quantization_params = input_details[0]['quantization_parameters']
    scale = quantization_params['scales'][0]
    zero_point = quantization_params['zero_points'][0]
    
    print(f"Quantizzazione input: scale={scale}, zero_point={zero_point}")
    
    # Quantizza l'input da float32 a int8
    input_data = np.round(input_data / scale + zero_point).astype(np.int8)
    print("Input quantizzato da float32 a int8")

# --- Imposta tensore input ---
interpreter.set_tensor(input_details[0]['index'], input_data)

# --- Esegui inferenza ---
interpreter.invoke()

# --- Ottieni output ---
output_data = interpreter.get_tensor(output_details[0]['index'])

# --- DEQUANTIZZAZIONE AUTOMATICA ---
if output_details[0]['dtype'] == np.int8:
    # Ottieni i parametri di quantizzazione dell'output
    quantization_params = output_details[0]['quantization_parameters']
    scale = quantization_params['scales'][0]
    zero_point = quantization_params['zero_points'][0]
        
    print(f"Dequantizzazione output: scale={scale}, zero_point={zero_point}")
        
    # Dequantizza l'output da int8 a float32
    output_data = (output_data.astype(np.float32) - zero_point) * scale
    print("Output dequantizzato da int8 a float32")

# Esperimenti sull'output a mano.
predictions = output_data[0] #output completo del modello
# Dopo aver ottenuto output_data
max_score = np.max(output_data)
min_score = np.min(output_data) 
print(f"Valore massimo nell'output: {max_score}")
print(f"Valore minimo nell'output: {min_score}")


# Verifica i valori per la classe 'person'
predictions = output_data[0]
person_scores = predictions[5, :]  # era indice 5
max_person_score = np.max(person_scores)
print(f"Valore massimo per classe 'person': {max_person_score}")

result = detect_person(output_data, confidence_threshold, iou_confidence) #0.000001 per float16/32
if result:
    detected_boxes, detected_scores = result
    nomeImmagine_preprocessed = nomeImmagine + "_preprocessed.jpg"
    # Disegna le bounding box sull'immagine preprocessata
    draw_bounding_boxes(nomeImmagine_preprocessed, detected_boxes, detected_scores)
else:
    print("Nessuna bounding box da disegnare")

Dettagli input modello: [{'name': 'images', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Dettagli output modello: [{'name': 'Identity', 'index': 10702, 'shape': array([   1,   84, 8400], dtype=int32), 'shape_signature': array([   1,   84, 8400], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
formato dell'input: <class 'numpy.float32'>
formato output: <class 'numpy.float32'>
Original image size: 640x480
Immagine preprocessata salvata come: Foto/example_preprocessed.jpg
Valore massimo nell'output: 0.995850265026092

In [ ]:
#Ridimensionamento delle immagini del dataset di calibrazione da 640x480 a 640x640
import os
import cv2

input_folder = "immagini_calibrazione640x480"
output_folder = "immagini_ridimensionate640x640"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".jpg"):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)  # dovrebbe essere 640x480

        if img is None:
            print(f"Impossibile leggere {filename}")
            continue
        
        h, w, _ = img.shape
        # Crop centrale quadrato 480x480
        side = min(h, w)  # 480
        start_x = (w - side) // 2  # (640-480)/2=80
        start_y = (h - side) // 2  # (480-480)/2=0

        img_cropped = img[start_y:start_y+side, start_x:start_x+side]

        # Resize a 640x640
        img_resized = cv2.resize(img_cropped, (640, 640))

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, img_resized)

        print(f"Elaborata {filename}")


In [ ]:
#Quantizzazione del modello YOLO11n in int8, con dataset di calibrazione
import tensorflow as tf
import numpy as np
import cv2
import os

# === Dataset di calibrazione ===
def representative_dataset_gen():
    folder = 'immagini_ridimensionate640x640'  # la cartella con le immagini 640x640
    image_paths = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    for path in image_paths:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0         # Normalizzazione [0, 1]
        img = np.expand_dims(img, axis=0)            # (1, 640, 640, 3)
        yield [img]

# === Conversione del modello ===
saved_model_dir = "yolo11n_saved_model"  # percorso del modello saved_model

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Imposto input e output in INT8 perché è quantizzazione full int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()

# === Salvataggio ===
output_path = "yolo11n_int8personal.tflite"
with open(output_path, "wb") as f:
    f.write(tflite_quant_model)

print(f"✅ Modello quantizzato salvato come {output_path}")


W0000 00:00:1754471551.205323 6422266 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1754471551.206203 6422266 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-06 11:12:31.209411: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: yolo11n_saved_model
2025-08-06 11:12:31.215852: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-06 11:12:31.215875: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: yolo11n_saved_model
2025-08-06 11:12:31.257055: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-06 11:12:31.360434: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: yolo11n_saved_model
2025-08-06 11:12:31.410747: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 201555 microseconds.
2025-08-06 11:12:31.874916: I tenso

✅ Modello quantizzato salvato come yolo11n_int8personal.tflite


In [ ]:
#TROVA TUTTE LE OPERAZIONI DEL MODELLO
import tensorflow as tf

nomeModello = "Modelli/yolo11n_full_integer_quant.tflite"

interpreter = tf.lite.Interpreter(model_path=nomeModello)
interpreter.allocate_tensors()

ops = set()
for d in interpreter._get_ops_details():
    ops.add(d['op_name'])

print("Operazioni usate nel modello:")
for op in sorted(ops):
    print(f"- {op}")
